<a href="https://colab.research.google.com/github/cgN77/opinion_mining/blob/main/TweetStuff_NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tweet Stuff


In [ ]:
import pdb
from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pandas as pd
import nltk
import string
from nltk.tokenize import TweetTokenizer
from os import getcwd
from nltk.stem import PorterStemmer
import re
from nltk.tokenize import word_tokenize
# import streamlit as st

In [ ]:
nltk.download('stopwords')
nltk.download('twitter_samples')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))
len(train_y)

8000

In [ ]:
stemmer = PorterStemmer()
def stem_words(words):
    return [stemmer.stem(word) for word in words]
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
  # words = text.split()
  return [lemmatizer.lemmatize(word) for word in text]

Tweet processing

In [ ]:
tw = "@FindBenNeedham it's my birthday today so for my birthday wish I hope there's good news about Ben soon :-)"

url_pattern = re.compile(r'https?://\S+')
# happy_emoticon=r':)'
# sad_emoticon=r':('

def remove_urls(text):
    return url_pattern.sub('', text)

def separate_common_word(compound_word, common_word):
    if common_word in compound_word:
        base_word, rest_of_word = compound_word.split(common_word, 1)
        return base_word+' '+common_word + rest_of_word
    else:
        return compound_word

def process_tweet(tweet):
  tweet=tweet.replace(':)','happyemoticon')
  tweet=separate_common_word(tweet,'happyemoticon')
  tweet=tweet.replace(':(','sademoticon')
  tweet=separate_common_word(tweet,'sademoticon')
  tweet=tweet.replace(':-)','happyemoticonwithnose')
  tweet=separate_common_word(tweet,'happyemoticonwithnose')
  tweet=tweet.replace(':-(','sademoticonwithnose')
  tweet=separate_common_word(tweet,'sademoticonwithnose')
  tweet=tweet.lower()
  tweet=separate_common_word(tweet,'d')
  tweet=remove_urls(tweet)
  tweet = re.sub(r'[^\w\s]', '', tweet)
  tweet=re.sub(r'\d','',tweet)
  tweet=word_tokenize(tweet)
  stop_words = set(stopwords.words('english'))
  tweet = [word for word in tweet if word.lower() not in stop_words]
  tweet=stem_words(tweet)
  tweet=lemmatize_words(tweet)
  return tweet

tw=process_tweet(tw)
tw

['fin',
 'dbenneedham',
 'birthday',
 'today',
 'birthday',
 'wish',
 'hope',
 'there',
 'good',
 'news',
 'ben',
 'soon',
 'happyemoticonwithnos']

Counting words

In [ ]:
def count_tweets(result,tweets,ys):
  for y,tweet in zip(ys,tweets):
    for word in process_tweet(tweet):
      pair=(word,y)
      if pair in result:
        result[pair]+=1
      else:
        result[pair]=1
  return result


In [ ]:
result = {}
tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
y = [1, 0, 0, 0, 0]
count_tweets(result, tweets, y)

{('happi', 1): 1, ('trick', 0): 1, ('sa', 0): 1, ('tire', 0): 2}

In [ ]:
frequencies = count_tweets({}, train_x, train_y)
frequencies[('happi',1)]

161

In [ ]:
def lookup(freqs,word,n):
  if (word,n) in freqs.keys():
    return freqs[(word,n)]
  return 0
lookup(frequencies,'town',1)

4

In [ ]:
def train_naive_bayes(frequencies,train_x,train_y):
  log_likelihood={}
  logprior=0

  vocab=set([pair[0] for pair in frequencies.keys()])
  v=len(vocab)

  Npos=Nneg=0
  for pair in frequencies.keys():
    if pair[1]>0:
      Npos+=frequencies[pair]
    else:
      Nneg+=frequencies[pair]

  D=len(train_y)
  Dpos=np.sum(train_y)
  Dneg=D-Dpos
  logprior=np.log(Dpos)-np.log(Dneg)

  for word in vocab:
    freq_pos=lookup(frequencies,word,1)
    freq_neg=lookup(frequencies,word,0)

    pwpos=(freq_pos+1)/(Npos+v)
    pwneg=(freq_neg+1)/(Nneg+v)

    log_likelihood[word]=np.log(pwpos/pwneg)

  return logprior,log_likelihood


In [ ]:
logprior,log_likelihood=train_naive_bayes(frequencies,train_x,train_y)
log_likelihood

{'mscarli': -0.7046309148364087,
 'step': -0.011483734276463417,
 'solvo': -0.7046309148364087,
 'elmoprinssi': 0.6816634462834819,
 'suprem': -0.011483734276463417,
 'til': -1.110096022944573,
 'standen': -0.7046309148364087,
 'loo': 0.6816634462834819,
 'win': -0.4814873635221989,
 'hacka': 0.6816634462834819,
 'nhe': -0.7046309148364087,
 'nbc': 0.6816634462834819,
 'amen': 1.0871285543916462,
 'supernatur': 0.6816634462834819,
 'papa': 0.6816634462834819,
 'oct': -0.41694884238462776,
 'danceeeee': -0.7046309148364087,
 'unfrien': -0.7046309148364087,
 'visa': 0.6816634462834819,
 'plane': -0.011483734276463417,
 'equalityact': 0.6816634462834819,
 'argoshelp': 0.6816634462834819,
 'partner': -1.110096022944573,
 'trash': -1.110096022944573,
 'gooood': 0.6816634462834819,
 'dvioletfilm': 0.6816634462834819,
 'neselec': 0.6816634462834819,
 'kanchu': -0.7046309148364087,
 'wolf': 1.5979541781576367,
 'maicaocampooo': -0.7046309148364087,
 'tyrafen': -0.7046309148364087,
 'evelinecon

In [ ]:
def naive_bayes_predict(tweet,logprior,log_likelihood):
  tweet=process_tweet(tweet)
  result=0
  for word in tweet:
    if word in log_likelihood.keys():
      result+=log_likelihood[word]
    else:
      result+=0
  result+=logprior
  return result

In [ ]:
t='i love this product'
naive_bayes_predict(t,logprior,log_likelihood)

4.327016704823454

In [ ]:
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great']:
  p = naive_bayes_predict(tweet, logprior, log_likelihood)
  print(f'{tweet} -> {p:.2f}')
my_tweet = 'you are bad :('
naive_bayes_predict(my_tweet, logprior, log_likelihood)

I am happy -> 2.13
I am bad -> -1.03
this movie should have been great. -> 2.28
great -> 2.13


-8.797732393495618

In [ ]:
print('Truth Predicted Tweet')
for x, y in zip(test_x, test_y):
    y_hat = naive_bayes_predict(x, logprior, log_likelihood)
    if y != (np.sign(y_hat) > 0):
        print('%d\t%0.2f\t%s' % (y, np.sign(y_hat) > 0, ' '.join(
            process_tweet(x)).encode('ascii', 'ignore')))

Truth Predicted Tweet
1	0.00	b'yoohoo shatter recor d bajrangibhaijaanstorm superhappi'
1	0.00	b'vvioletta highfiv probabl ahea sinc le artsi verbal'
1	0.00	b'dat rp tho thank much guy celebr one month partnership ty madmorphtv raid'
1	0.00	b'seniorspazz tehsmiley bore everyth'
1	0.00	b'jare dnotsubway iluvmariah bravotv truli later move know queen bee upward bound movingonup'
1	0.00	b'disonivi wish ddddddd kid'
1	0.00	b'catargiu yeah kin da feel like warm butter'
1	0.00	b'gotzefi im tri dl mh english patch psp'
1	0.00	b'_sw_e_ safe'
1	0.00	b'new report talk burn calori col work harder warm feel better weather p'
1	0.00	b'shutupbricklp uhc box larg scale one network server p'
1	0.00	b'vincenttong cant wait week'
1	0.00	b'tbhhowel oh shoot well watch'
1	0.00	b'spaqhetto artofbs littl finger'
1	0.00	b'happyemoticonhappyemoticonhappyemoticonhappyemoticon'
1	0.00	b'hahan thousand lie'
1	0.00	b'fallout get see glow sea technic fallout play see'
1	0.00	b'harri niall harri born ik stupid wan 

In [ ]:
def accuracy(test_x, test_y, logprior, log_likelihood):
    y_pred=[]
    l=[]
    for tweet,y in zip(test_x, test_y):
        prediction = 1 if naive_bayes_predict(tweet, logprior, log_likelihood) > 0 else 0
        y_pred.append(prediction)
        l.append(1 if prediction==y else 0)
    return np.sum(l)/len(l)*100

In [ ]:
accuracy(test_x,test_y,logprior,log_likelihood)

97.2

In [ ]:
tweet='i am good :)'
x=naive_bayes_predict(tweet,logprior,log_likelihood)
if x>1:
    print(':)')
elif x<1 and x>-1:
    print(':|')
else:
    print(':)')

:)
